<a href="https://colab.research.google.com/github/KoMurase/Learning_my_interest/blob/master/%E6%8E%A8%E8%96%A6%E3%82%B7%E3%82%B9%E3%83%86%E3%83%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget http://files.grouplens.org/datasets/movielens/ml-20m.zip

--2019-12-25 09:00:58--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  12.6MB/s    in 17s     

2019-12-25 09:01:20 (11.5 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]

--2019-12-25 09:01:23--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip.1’

ml-20m.zip.1        100%[===================>] 189.50M  14.1MB/s    in 15s     

2019-12-25 09:01:38 (12.4 MB/s) - ‘ml-20m.zip.1’

In [4]:
!unzip ml-20m.zip

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


In [0]:
import pandas as pd 
genome_scores=pd.read_csv('ml-20m/genome-scores.csv')
genome_tags = pd.read_csv('/content/ml-20m/genome-tags.csv')
links = pd.read_csv('/content/ml-20m/links.csv')
movies = pd.read_csv('/content/ml-20m/movies.csv')
ratings = pd.read_csv('/content/ml-20m/ratings.csv')
tags = pd.read_csv('/content/ml-20m/tags.csv')

In [19]:
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [20]:
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [21]:
len(ratings)

20000263

In [22]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [0]:
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset,DataLoader, Dataset
import tqdm 
from sklearn.model_selection import train_test_split

X = ratings[["userId","movieId"]].values
Y = ratings["rating"].values

train_x, test_x, train_y,test_y = train_test_split(X,Y,test_size=0.1)

train_dataset = TensorDataset(torch.tensor(train_x, dtype=torch.int64),
                        torch.tensor(train_y, dtype=torch.float32))

test_dataset = TensorDataset(torch.tensor(test_x, dtype=torch.int64),
                       torch.tensor(test_y, dtype=torch.float32))

#num_workersでいくつのコアでデータをロードするか指定(デフォルトはメインのみ)
train_loader = DataLoader(train_dataset, batch_size=1024, num_workers=4,shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=1024, num_workers=4,shuffle=False)

<h3>ユーザーや商品のIDをK次元ベクトルに変換する</h3>


In [0]:
class MetricsFactorization(nn.Module):
  def __init__(self, max_user, max_item, k=20):
    super().__init__()
    self.max_user = max_user
    self.max_item = max_item
    self.user_emb = nn.Embedding(max_user, k, 0)
    self.item_emb = nn.Embedding(max_item, k, 0)
  
  def forward(self, x):
    user_idx = x[:, 0]
    item_idx = x[:, 1]
    user_feature = self.user_emb(user_idx)
    item_feature = self.item_emb(item_idx)

    #user_feature*item_featureは(batch_size, k)次元なので
    #kについてsumをとるとそれぞれのサンプルの内積になる
    out = torch.sum(user_feature * item_feature, 1)

    #単純に内積をそのまま評価として予測値に使用するとはみ出すため
    #[0, 5]の範囲に収まるように変換
    out = nn.functional.sigmoid(out) * 5
    return out

In [25]:
X.max(0)

array([138493, 131262])

In [0]:
max_user,max_item=X.max(0)
#np.int64型をPythonの標準のintにキャスト
max_user = int(max_user)
max_item = int(max_item)
net = MetricsFactorization(max_user+1, max_item+1)

In [0]:
#MAEを使って評価関数を作る

def eval_net(net, loader, score_fn=nn.functional.l1_loss, device="cpu"):
  ys = []
  ypreds = []
  for x,y in loader:
    x = x.to(device)
    ys.append(y)
    with torch.no_grad(): 
      ypred = net(x).to("cpu").view(-1)
      ypreds.append(ypred)
    score = score_fn(torch.cat(ys).squeeze(), torch.cat(ypreds))
  return score.item()

In [38]:
#訓練部分の作成
from statistics import mean 

net.to("cuda:0")
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()

for epoch in range(5):
  loss_log=[]
  for x, y in tqdm.tqdm(train_loader):
    x = x.to("cuda:0")
    y = y.to("cuda:0")
    o = net(x)
    loss = loss_f(o, y.view(-1))
    net.zero_grad()
    loss.backward()
    opt.step()
    loss_log.append(loss.item())
  test_score = eval_net(net, test_loader, device="cuda:0")
  print(epoch, mean(loss_log), test_score, flush=True)

#訓練したモデルをcpuに移す
net.to("cpu")
#ユーザ1の映画10の評価を計算したい
query = (1,10)
#int64のTensorに変換し、batchの次元を付加
query = torch.tensor(query, dtype=torch.int64).view(1, -1)

#netに渡す
net(query)

  0%|          | 0/17579 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 17579/17579 [03:06<00:00, 94.38it/s]


KeyboardInterrupt: ignored

In [0]:
query = torch.stack([
                     torch.zeros(max_item).fill_(1),
                     torch.arange(1, max_item+1)],1).long()
#scoresは上位k本のスコア
#indicesmは上位k本の位置, すなわちmovieId
scores, indices = torch.topk(net(query), 5)